In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree, svm, metrics
from sklearn.metrics import r2_score

In [ ]:
dataset = pd.read_csv('Onion_Prices_2020_preprocessed.csv')

In [ ]:
dataset.head()

,state_cl,district_cl,market_cl,variety_cl,arrival_date_cl,min_price,max_price,modal_price
0,19,178,486,3,18,1350,4390,3100.0
1,19,178,486,3,27,1390,4400,3200.0
2,19,178,486,3,45,1460,5150,4310.0
3,19,178,486,3,54,2010,5200,4200.0
4,19,178,486,3,81,1320,4050,3300.0


In [ ]:
dataset.describe()

,state_cl,district_cl,market_cl,variety_cl,arrival_date_cl,min_price,max_price,modal_price
count,107295.000000,107295.000000,107295.000000,107295.000000,107295.000000,107295.000000,107295.000000,107295.000000
mean,3.618733,147.068680,435.216627,1.548646,129.989561,1894.479715,2289.960492,2109.523217
std,4.141079,89.518145,264.848203,2.242373,73.342018,1459.415839,1565.827631,1493.344338
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
25%,0.000000,75.000000,203.000000,0.000000,67.000000,1000.000000,1220.000000,1150.000000
50%,2.000000,143.000000,430.000000,1.000000,130.000000,1400.000000,1800.000000,1600.000000
75%,6.000000,223.000000,667.000000,2.000000,192.000000,2400.000000,2800.000000,2550.000000
max,21.000000,314.000000,904.000000,20.000000,259.000000,18000.000000,25000.000000,22000.000000


In [ ]:
for x in ['state_cl','variety_cl', 'min_price', 'max_price', 'modal_price']:
    q75,q25 = np.percentile(dataset.loc[:,x],[75,25]) # первый и третий квартиль
    intr_qr = q75-q25 # оценка межквартильного диапазона

    max = q75+(1.5*intr_qr)
    min = q25-(1.5*intr_qr)

    dataset.loc[dataset[x] < min,x] = np.nan # оценка нижней границы
    dataset.loc[dataset[x] > max,x] = np.nan # оценка верхней границы

In [ ]:
dataset = dataset.dropna(axis = 0)

In [ ]:
dataset.describe()

,state_cl,district_cl,market_cl,variety_cl,arrival_date_cl,min_price,max_price,modal_price
count,92845.000000,92845.000000,92845.000000,92845.000000,92845.000000,92845.000000,92845.000000,92845.000000
mean,3.449286,147.224665,437.658818,1.058054,132.626657,1596.352189,1954.067543,1790.616895
std,3.938235,90.150676,266.277462,1.215525,72.826898,906.184891,983.226393,919.306820
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
25%,0.000000,73.000000,203.000000,0.000000,71.000000,1000.000000,1200.000000,1100.000000
50%,2.000000,141.000000,428.000000,1.000000,134.000000,1300.000000,1600.000000,1500.000000
75%,6.000000,223.000000,680.000000,1.000000,195.000000,2025.000000,2500.000000,2300.000000
max,15.000000,314.000000,904.000000,5.000000,259.000000,4500.000000,5165.000000,4650.000000


In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset["modal_price"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train

array([[1.000e+00, 9.500e+01, 2.580e+02, ..., 5.500e+01, 2.000e+03,
        3.000e+03],
       [2.000e+00, 1.890e+02, 5.350e+02, ..., 2.480e+02, 4.000e+03,
        4.100e+03],
       [0.000e+00, 1.580e+02, 3.800e+02, ..., 2.050e+02, 8.000e+02,
        1.000e+03],
       ...,
       [1.000e+00, 2.670e+02, 5.200e+02, ..., 3.400e+01, 1.000e+03,
        1.200e+03],
       [6.000e+00, 3.000e+00, 4.350e+02, ..., 6.900e+01, 1.340e+03,
        1.425e+03],
       [0.000e+00, 1.190e+02, 7.790e+02, ..., 6.600e+01, 1.700e+03,
        1.800e+03]])

# Регрессия

In [ ]:
params = {
    'n_jobs':np.arange(0, 30),
    'positive':["True", "False"]
}
lr = LinearRegression()
l = GridSearchCV(lr, params)
l.fit(X_train, y_train)
l.best_params_

{'n_jobs': 0, 'positive': 'True'}

In [ ]:
l_with = LinearRegression(n_jobs=0, positive=True)
l_with.fit(X_train, y_train)

l_without = LinearRegression()
l_without.fit(X_train, y_train)

LinearRegression()

In [ ]:
print("Accuracy:", l_with.score(X_test, y_test),"c наилучшими гиперепараметрами")
print("Accuracy:", l_without.score(X_test, y_test),"без гиперепараметров")

Accuracy: 0.9777466578477174 c наилучшими гиперепараметрами
Accuracy: 0.9777679963930895 без гиперепараметров


In [ ]:
y_pred= l_without.predict(X_test)

R2 = metrics.r2_score(y_test, y_pred)
MAE = metrics.mean_absolute_error(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

print('R2:', R2)
print('MAE:', MAE)
print('MSE:', MSE)
print('RMSE:', RMSE)

R2: 0.9777679963930895
MAE: 52.7521565596511
MSE: 18811.10242279658
RMSE: 137.15357240260488


# Регрессия дерева решений

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)

DecisionTreeRegressor()

In [ ]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
66768,1120.0,1115.876544
104900,1700.0,1694.244014
43537,700.0,905.745939
67127,1260.0,1263.267974
1172,900.0,903.878779
...,...,...
104668,2100.0,2159.233809
80146,1150.0,1153.450036
61065,2750.0,2758.596027
55055,1250.0,1529.873661


In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 34.84694383111638
Mean Squared Error: 10838.752951155151
Root Mean Squared Error: 104.10933171985666


# Lasso

In [ ]:
# https://habr.com/ru/company/vk/blog/513842/
from sklearn.linear_model import LassoCV
regressor = LassoCV()
regressor.fit(X_train, y_train)

LassoCV()

In [ ]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
66768,1120.0,1120.0
104900,1700.0,1700.0
43537,700.0,800.0
67127,1260.0,1260.0
1172,900.0,850.0
...,...,...
104668,2100.0,2200.0
80146,1150.0,1150.0
61065,2750.0,2750.0
55055,1250.0,1500.0


In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 52.718991410586895
Mean Squared Error: 18840.00742584861
Root Mean Squared Error: 137.258906544707


# Гребневая регрессия

In [ ]:
# https://habr.com/ru/company/vk/blog/513842/
from sklearn.linear_model import RidgeCV
regressor = RidgeCV()
regressor.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [ ]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
66768,1120.0,1120.275417
104900,1700.0,1695.736723
43537,700.0,905.078257
67127,1260.0,1263.207483
1172,900.0,914.410865
...,...,...
104668,2100.0,2155.732653
80146,1150.0,1153.075159
61065,2750.0,2763.727191
55055,1250.0,1538.405507


In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 52.752346891510925
Mean Squared Error: 18811.10161258455
Root Mean Squared Error: 137.1535694489376


# Elastic Net регрессия

In [ ]:
# https://habr.com/ru/company/vk/blog/513842/
from sklearn.linear_model import ElasticNetCV
regressor = ElasticNetCV()
regressor.fit(X_train, y_train)

ElasticNetCV()

In [ ]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
66768,1120.0,1115.877099
104900,1700.0,1694.244133
43537,700.0,905.746701
67127,1260.0,1263.268357
1172,900.0,903.879650
...,...,...
104668,2100.0,2159.233445
80146,1150.0,1153.450546
61065,2750.0,2758.595450
55055,1250.0,1529.873872


In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 52.82262194549752
Mean Squared Error: 18839.066653713184
Root Mean Squared Error: 137.25547950341794


# task 2

In [ ]:
X = dataset.iloc[:, :-1].values
y = dataset["modal_price"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Линейная регрессия

In [ ]:
def Matrix(X, Y):
  return np.dot(X, Y)

In [ ]:
class LinReg():
    def __init__(self, X, y, alpha=0.03, n_iter=1500):

        self.alpha = alpha
        self.n_iter = n_iter
        self.n_samples = len(y)
        self.n_features = np.size(X, 1)
        self.X = np.hstack((np.ones(
            (self.n_samples, 1)), (X - np.mean(X, 0)) / np.std(X, 0))) #
        self.y = y[:, np.newaxis]
        self.params = np.zeros((self.n_features + 1, 1))
        self.coef_ = None
        self.intercept_ = None

    #Реализация Градиентного спуска
    def fit(self):
        for i in range(self.n_iter):
            #антиградиент
            self.params = self.params - (self.alpha/self.n_samples) * \
            Matrix(self.X.T, (Matrix(self.X, self.params) - self.y))
        
        self.intercept_ = self.params[0]
        self.coef_ = self.params[1:]

        return self

    def score(self, X=None, y=None):

        n_samples = np.size(X, 0)
        X = np.hstack((np.ones(
                (n_samples, 1)), (X - np.mean(X, 0)) / np.std(X, 0)))

        y = y[:, np.newaxis]

        y_pred = Matrix(X, self.params)
        score = 1 - (((y - y_pred)**2).sum() / ((y - y.mean())**2).sum())

        return score

    def predict(self, X):
        n_samples = np.size(X, 0)
        y = Matrix(np.hstack((np.ones((n_samples, 1)), (X-np.mean(X, 0)) \
                            / np.std(X, 0))), self.params)
        return y

    def get_params(self):

        return self.params

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd

our_regressor = LinReg(X_train, y_train).fit()
sklearn_regressor = LinearRegression().fit(X_train, y_train)

our_train_accuracy = our_regressor.score(X_train, y_train) #точность обучающей выборки
sklearn_train_accuracy = sklearn_regressor.score(X_train, y_train)

our_test_accuracy = our_regressor.score(X_test, y_test) #точность тестовой выборки
sklearn_test_accuracy = sklearn_regressor.score(X_test, y_test)

pd.DataFrame([[our_train_accuracy, sklearn_train_accuracy],
              [our_test_accuracy, sklearn_test_accuracy]],
             ['Training Accuracy', 'Test Accuracy'],    
             ['Our Implementation', 'Sklearn\'s Implementation'])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


,Our Implementation,Sklearn's Implementation
Training Accuracy,0.980582,0.980582
Test Accuracy,0.977764,0.977768


In [ ]:
y_pred = our_regressor.predict(X_test)

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 52.51404524504298
Mean Squared Error: 18814.555757340415
Root Mean Squared Error: 137.16616112343604
